# Задатак класификације текста

У овом модулу ћемо започети са једноставним задатком класификације текста заснованим на скупу података **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)**: класификоваћемо наслове вести у једну од 4 категорије: Свет, Спорт, Бизнис и Наука/Технологија.

## Скуп података

За учитавање скупа података користићемо **[TensorFlow Datasets](https://www.tensorflow.org/datasets)** API.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Сада можемо приступити деловима скупа података за обуку и тестирање користећи `dataset['train']` и `dataset['test']` респективно:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Хајде да одштампамо првих 10 нових наслова из нашег скупа података:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Векторизација текста

Сада треба да претворимо текст у **бројеве** који могу бити представљени као тензори. Ако желимо репрезентацију на нивоу речи, потребно је да урадимо две ствари:

* Користимо **токенизатор** за раздвајање текста на **токене**.
* Направимо **речник** тих токена.

### Ограничење величине речника

У примеру са AG News скупом података, величина речника је прилично велика, више од 100 хиљада речи. Генерално гледано, не требају нам речи које се ретко појављују у тексту — само неколико реченица ће их садржати, а модел их неће научити. Због тога има смисла ограничити величину речника на мањи број тако што ћемо проследити аргумент конструктору векторизатора:

Оба ова корака могу се обавити коришћењем слоја **TextVectorization**. Хајде да инстанцирамо објекат векторизатора, а затим позовемо метод `adapt` како бисмо прошли кроз цео текст и направили речник:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Напомена** да користимо само подскуп целокупног скупа података за изградњу речника. То радимо како бисмо убрзали време извршења и не бисмо вас држали у ишчекивању. Међутим, преузимамо ризик да неке речи из целокупног скупа података не буду укључене у речник и да буду игнорисане током обуке. Стога, коришћење целокупне величине речника и пролазак кроз цео скуп података током `adapt` би требало да повећа коначну тачност, али не значајно.

Сада можемо приступити стварном речнику:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Користећи векторизатор, можемо лако кодирати било који текст у скуп бројева:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Представљање текста методом "торба речи"

Пошто речи представљају значење, понекад можемо схватити значење неког текста само гледајући појединачне речи, без обзира на њихов редослед у реченици. На пример, приликом класификације вести, речи као што су *време* и *снег* вероватно указују на *временску прогнозу*, док би речи као што су *акције* и *долар* биле повезане са *финансијским вестима*.

**Торба речи** (BoW) представљање вектора је најједноставније за разумевање међу традиционалним представљањима вектора. Свака реч је повезана са индексом вектора, а елемент вектора садржи број појављивања сваке речи у датом документу.

![Слика која приказује како је представљање вектора методом "торба речи" приказано у меморији.](../../../../../translated_images/sr/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Можете такође размишљати о BoW као о збиру свих једноврсно-кодираних вектора за појединачне речи у тексту.

Испод је пример како да генеришете представљање методом "торба речи" користећи Python библиотеку Scikit Learn:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Можемо такође користити Керас векторизатор који смо дефинисали изнад, претварајући сваки број речи у one-hot кодирање и сабирајући све те векторе:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Напомена**: Можда ћете бити изненађени што се резултат разликује од претходног примера. Разлог је тај што у примеру са Keras-ом дужина вектора одговара величини речника, који је направљен из целог AG News скупа података, док смо у примеру са Scikit Learn-ом речник изградили директно из узорка текста.


## Тренирање BoW класификатора

Сада када смо научили како да изградимо представу текста у облику "вреће речи" (bag-of-words), хајде да обучимо класификатор који је користи. Прво, потребно је да наш скуп података конвертујемо у представу "вреће речи". Ово можемо постићи коришћењем функције `map` на следећи начин:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Сада хајде да дефинишемо једноставну класификаторску неуронску мрежу која садржи један линеарни слој. Улазна величина је `vocab_size`, а излазна величина одговара броју класа (4). Пошто решавамо задатак класификације, завршна активациона функција је **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Пошто имамо 4 класе, тачност изнад 80% је добар резултат.

## Тренирање класификатора као једне мреже

Пошто је векторизатор такође Keras слој, можемо дефинисати мрежу која га укључује и тренирати је од почетка до краја. На овај начин не морамо да векторизујемо скуп података користећи `map`, већ можемо једноставно проследити оригинални скуп података на улаз мреже.

> **Напомена**: И даље бисмо морали да применимо `map` на наш скуп података како бисмо конвертовали поља из речника (као што су `title`, `description` и `label`) у парове. Међутим, када учитавамо податке са диска, можемо одмах направити скуп података са потребном структуром.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Биграми, триграми и n-грамови

Једно од ограничења приступа "вреће речи" је то што су неке речи део израза састављених од више речи. На пример, реч 'hot dog' има потпуно другачије значење од речи 'hot' и 'dog' у другим контекстима. Ако речи 'hot' и 'dog' увек представљамо истим векторима, то може збунити наш модел.

Да бисмо ово решили, често се користе **представе n-грамова** у методама класификације докумената, где је учесталост сваке речи, двословне или трословне комбинације корисна карактеристика за тренирање класификатора. У представама биграма, на пример, додајемо све парове речи у речник, поред оригиналних речи.

Испод је пример како да генеришете представу "вреће речи" са биграмима користећи Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Главни недостатак n-грам приступа је што величина речника почиње да расте изузетно брзо. У пракси, потребно је комбиновати n-грам репрезентацију са техником смањења димензионалности, као што су *уграђивања* (*embeddings*), о чему ћемо говорити у наредној јединици.

Да бисмо користили n-грам репрезентацију у нашем **AG News** скупу података, потребно је да проследимо параметар `ngrams` нашем конструктору `TextVectorization`. Дужина речника биграма је **значајно већа**, у нашем случају више од 1,3 милиона токена! Због тога има смисла ограничити број биграм токена на неки разумни број.

Могли бисмо користити исти код као горе за тренирање класификатора, али би то било веома неефикасно у погледу меморије. У наредној јединици ћемо тренирати класификатор биграма користећи уграђивања. У међувремену, можете експериментисати са тренирањем класификатора биграма у овом бележнику и видети да ли можете постићи већу тачност.


## Аутоматско израчунавање BoW вектора

У горњем примеру смо ручно израчунавали BoW векторе сабирањем one-hot кодирања појединачних речи. Међутим, најновија верзија TensorFlow-а нам омогућава да аутоматски израчунамо BoW векторе тако што проследимо параметар `output_mode='count` конструктору векторизатора. Ово значајно олакшава дефинисање и тренирање нашег модела:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Учесталост термина - инверзна учесталост докумената (TF-IDF)

У представљању BoW, појаве речи се вреднују истом техником без обзира на саму реч. Међутим, јасно је да су честе речи као што су *и* и *у* много мање важне за класификацију од специјализованих термина. У већини задатака обраде природног језика (NLP) неке речи су релевантније од других.

**TF-IDF** означава **учесталост термина - инверзна учесталост докумената**. То је варијација модела "вреће речи" (bag-of-words), где се уместо бинарне вредности 0/1, која указује на појаву речи у документу, користи вредност са покретним зарезом, која је повезана са учесталошћу појаве речи у корпусу.

Формалније, тежина $w_{ij}$ речи $i$ у документу $j$ дефинисана је као:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
где су:
* $tf_{ij}$ број појава речи $i$ у документу $j$, односно BoW вредност коју смо раније видели
* $N$ број докумената у збирци
* $df_i$ број докумената који садрже реч $i$ у целој збирци

TF-IDF вредност $w_{ij}$ расте пропорционално броју појава речи у документу и смањује се у зависности од броја докумената у корпусу који садрже ту реч, што помаже у прилагођавању чињеници да се неке речи чешће појављују од других. На пример, ако се реч појављује у *сваком* документу у збирци, $df_i=N$, и $w_{ij}=0$, те би те речи биле потпуно занемарене.

TF-IDF векторизацију текста можете лако креирати помоћу Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

У Керасу, слој `TextVectorization` може аутоматски израчунати TF-IDF фреквенције прослеђивањем параметра `output_mode='tf-idf'`. Хајде да поновимо код који смо користили изнад да видимо да ли употреба TF-IDF повећава тачност:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Закључак

Иако TF-IDF репрезентације додељују тежине фреквенције различитим речима, оне нису у стању да представе значење или редослед. Као што је чувени лингвиста Џ. Р. Фирт рекао 1935. године: „Потпуно значење речи је увек контекстуално, и ниједно проучавање значења ван контекста не може се сматрати озбиљним.“ Касније у курсу ћемо научити како да ухватимо контекстуалне информације из текста користећи језичко моделирање.



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да превод буде тачан, молимо вас да имате у виду да аутоматизовани преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати ауторитативним извором. За критичне информације препоручује се професионални превод од стране људи. Не преузимамо одговорност за било каква погрешна тумачења или неспоразуме који могу настати услед коришћења овог превода.
